In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn import metrics
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [2]:
data = pd.read_csv("bank.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data2 = data.drop(["RowNumber", "CustomerId", "Surname"], axis=1)
data2.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data2.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


There appear to be no obvious outliers to worry about

In [5]:
data_encoded = pd.get_dummies(data2 , columns=['Geography','Gender'])
data_encoded.transpose()

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
CreditScore,619.00,608.00,502.00,699.00,850.00,645.00,822.0,376.00,501.00,684.00,...,714.00,597.00,726.0,644.00,800.00,771.00,516.00,709.00,772.00,792.00
Age,42.00,41.00,42.00,39.00,43.00,44.00,50.0,29.00,44.00,27.00,...,33.00,53.00,36.0,28.00,29.00,39.00,35.00,36.00,42.00,28.00
Tenure,2.00,1.00,8.00,1.00,2.00,8.00,7.0,4.00,4.00,2.00,...,3.00,4.00,2.0,7.00,2.00,5.00,10.00,7.00,3.00,4.00
Balance,0.00,83807.86,159660.80,0.00,125510.82,113755.78,0.0,115046.74,142051.07,134603.88,...,35016.60,88381.21,0.0,155060.41,0.00,0.00,57369.61,0.00,75075.31,130142.79
NumOfProducts,1.00,1.00,3.00,2.00,1.00,2.00,2.0,4.00,2.00,1.00,...,1.00,1.00,1.0,1.00,2.00,2.00,1.00,1.00,2.00,1.00
HasCrCard,1.00,0.00,1.00,0.00,1.00,1.00,1.0,1.00,0.00,1.00,...,1.00,1.00,1.0,1.00,0.00,1.00,1.00,0.00,1.00,1.00
IsActiveMember,1.00,1.00,0.00,0.00,1.00,0.00,1.0,0.00,1.00,1.00,...,0.00,0.00,0.0,0.00,0.00,0.00,1.00,1.00,0.00,0.00
EstimatedSalary,101348.88,112542.58,113931.57,93826.63,79084.10,149756.71,10062.8,119346.88,74940.50,71725.73,...,53667.08,69384.71,195192.4,29179.52,167773.55,96270.64,101699.77,42085.58,92888.52,38190.78
Exited,1.00,0.00,1.00,0.00,0.00,1.00,0.0,1.00,0.00,0.00,...,0.00,1.00,0.0,0.00,0.00,0.00,0.00,1.00,1.00,0.00
Geography_France,1.00,0.00,1.00,1.00,0.00,0.00,1.0,0.00,1.00,1.00,...,0.00,1.00,0.0,1.00,1.00,1.00,1.00,1.00,0.00,1.00


In [6]:
data_encoded.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [7]:
from sklearn.model_selection import train_test_split

X = data_encoded.drop(["Exited"], axis=1)
Y = data_encoded["Exited"]
X_train, X_test, ytrain, ytest = train_test_split( X, Y, test_size = 0.3, random_state = 42 )


In [8]:
from tensorflow import keras

y_train = keras.utils.to_categorical(ytrain, 10)
y_test = keras.utils.to_categorical(ytest, 10)

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)

In [10]:
model = Sequential()
model.add(Dense(784,activation='relu'))
model.add(Dense(100, activation ='relu'))
model.add(Dense(10,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=256, verbose=1) 
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.3f'  % acc)
print('Loss: %.3f' % loss)



Epoch 1/20
28/28 [==============================] - 0s 2ms/step - loss: 0.8602 - accuracy: 0.7534
Epoch 2/20
28/28 [==============================] - 0s 2ms/step - loss: 0.4130 - accuracy: 0.8246
Epoch 3/20
28/28 [==============================] - 0s 2ms/step - loss: 0.3815 - accuracy: 0.8403
Epoch 4/20
28/28 [==============================] - 0s 2ms/step - loss: 0.3587 - accuracy: 0.8541
Epoch 5/20
28/28 [==============================] - 0s 2ms/step - loss: 0.3496 - accuracy: 0.8536
Epoch 6/20
28/28 [==============================] - 0s 3ms/step - loss: 0.3424 - accuracy: 0.8591
Epoch 7/20
28/28 [==============================] - 0s 2ms/step - loss: 0.3403 - accuracy: 0.8571
Epoch 8/20
28/28 [==============================] - 0s 2ms/step - loss: 0.3369 - accuracy: 0.8589
Epoch 9/20
28/28 [==============================] - 0s 3ms/step - loss: 0.3352 - accuracy: 0.8601
Epoch 10/20
28/28 [==============================] - 0s 2ms/step - loss: 0.3307 - accuracy: 0.8650
Epoch 11/20
28/28 [

## According to the above and after running this a few times, on average about 10 epochs is ideal and we should stick with 10 epochs because we don't gain much after that

In [11]:
model2 = Sequential()
model2.add(Dense(784,activation='relu'))
model2.add(Dense(100, activation ='relu'))
model2.add(Dense(10,activation='softmax'))

model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model2.fit(X_train, y_train, epochs=10, batch_size=256, verbose=1) 
loss2, acc2 = model2.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.3f'  % acc2)
print('Loss: %.3f' % loss2)

Epoch 1/10
28/28 [==============================] - 0s 2ms/step - loss: 0.8628 - accuracy: 0.7627
Epoch 2/10
28/28 [==============================] - 0s 2ms/step - loss: 0.4247 - accuracy: 0.8214
Epoch 3/10
28/28 [==============================] - 0s 2ms/step - loss: 0.3958 - accuracy: 0.8353
Epoch 4/10
28/28 [==============================] - 0s 2ms/step - loss: 0.3725 - accuracy: 0.8437
Epoch 5/10
28/28 [==============================] - 0s 2ms/step - loss: 0.3575 - accuracy: 0.8523
Epoch 6/10
28/28 [==============================] - 0s 2ms/step - loss: 0.3493 - accuracy: 0.8537
Epoch 7/10
28/28 [==============================] - 0s 2ms/step - loss: 0.3427 - accuracy: 0.8570
Epoch 8/10
28/28 [==============================] - 0s 2ms/step - loss: 0.3390 - accuracy: 0.8610
Epoch 9/10
28/28 [==============================] - 0s 2ms/step - loss: 0.3358 - accuracy: 0.8601
Epoch 10/10
28/28 [==============================] - 0s 3ms/step - loss: 0.3324 - accuracy: 0.8609
Accuracy: 0.867
Los

In [12]:
y_predict = model.predict(X_test)
y_predict[0]


array([9.7581488e-01, 2.4077745e-02, 4.7244935e-06, 2.2070550e-05,
       8.8145034e-06, 2.2726757e-05, 1.3199506e-05, 8.4839585e-06,
       1.3600643e-05, 1.3762105e-05], dtype=float32)

In [13]:
y_pred = []
for val in y_predict:
    y_pred.append(np.argmax(val))
#print(y_pred)    
#convert 0 1 to 1 and 1 0 as 0
cm = metrics.confusion_matrix(ytest,y_pred)
print(cm)

[[2357   59]
 [ 343  241]]


In [14]:
cr=metrics.classification_report(ytest,y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.87      0.98      0.92      2416
           1       0.80      0.41      0.55       584

    accuracy                           0.87      3000
   macro avg       0.84      0.69      0.73      3000
weighted avg       0.86      0.87      0.85      3000



## For threshold of 0.5, here are the metrics

In [15]:
testing_pred = (y_predict > 0.5).astype(int)

y_pred_0_5 = []
for val in testing_pred:
    y_pred_0_5.append(np.argmax(val))
#print(y_pred)    
#convert 0 1 to 1 and 1 0 as 0
cm = metrics.confusion_matrix(ytest,y_pred_0_5)
print(cm)

[[2357   59]
 [ 343  241]]


In [16]:
cr=metrics.classification_report(ytest,y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.87      0.98      0.92      2416
           1       0.80      0.41      0.55       584

    accuracy                           0.87      3000
   macro avg       0.84      0.69      0.73      3000
weighted avg       0.86      0.87      0.85      3000



This threshold gives the same metrics back so the threshold chosen by the model must have been 0.5 or extremely close to 0.5